In [1]:
#Generation of learning curve data

import utils
from numpy.linalg import lstsq

import sklearn
from sklearn.metrics import mean_squared_error

import pandas as pd
from pandas import DataFrame

import matplotlib.pyplot as plt



In [9]:
def model_accuracy(cat, cut, smo_cut, sp_size, ker_exp, reg, tr_size):

    keys = utils.keys_grabber(cat)
    #keys = [2]
    soap = utils.descriptor(cut, smo_cut)
    full_spec, iso= utils.puller(keys, soap, cat)
    
    sp_sp, tr_sp, tr_ta, te_sp, te_ta = utils.splitter(tr_size, sp_size, full_spec, iso)
    
    KNM = utils.kerneller(tr_sp, sp_sp, ker_exp)
    KMM = utils.kerneller(sp_sp, sp_sp, ker_exp)
    KTM = utils.kerneller(te_sp, sp_sp, ker_exp)
    res = lstsq(KNM.T @ KNM + reg * KMM, KNM.T @ tr_ta, rcond=None)
    c = res[0]
    pred = KTM @ c
    rmse = mean_squared_error(te_ta, pred, squared=False)

    
    return te_ta, pred, rmse

def species_splitter(target, predicted):
    ids_o = []
    ids_si = []

    for i in enumerate(target):
        if i[1] < 350:
            ids_o.append(i[0])
        else:
            ids_si.append(i[0])
            
    o_tar = target[ids_o]
    o_pred = predicted[ids_o]
    
    si_tar = target[ids_si]
    si_pred = predicted[ids_si]
    
    return o_tar, si_tar, o_pred, si_pred

In [58]:
#Rattle
cat = 'AM300K'
cut = 3.5
smo_cut = 1
sp_size = 1000
ker_exp = 25
reg = 1*10**-8

tr_sizes = range(1000,6001,1000)
o_rmses = []
si_rmses = []



for i in tr_sizes:
    print(i)
    counter = 0
    temp_o = []
    temp_si = []
    
    while counter < 25:   

        target, predicted, rmse = model_accuracy(cat, cut, smo_cut, sp_size, ker_exp, reg, i)
        o_tar, si_tar, o_pred, si_pred = species_splitter(target, predicted)

        temp_o.append(mean_squared_error(o_tar, o_pred, squared=False))
        
        temp_si.append(mean_squared_error(si_tar, si_pred, squared=False))
        counter += 1
        
   
    o_rmses.append(sum(temp_o)/len(temp_o))
    si_rmses.append(sum(temp_si)/len(temp_si))

df = pd.DataFrame()
df['train sizes'] = tr_sizes
df['o rmse'] = o_rmses
df['si rmse'] = si_rmses
df.to_csv('Learning_curves_split_' +cat+'.csv')

1000
2000
3000
4000
5000
6000


In [3]:
#Rattle
cat = 'HypoZeo'
cut = 3.5
smo_cut = 1
sp_size = 1000
ker_exp = 25
reg = 1*10**-8
target, predicted, rmse = model_accuracy('HypoZeo', cut, 1, sp_size, ker_exp, reg)
print(mean_squared_error(target,predicted, squared=False))
print(rmse)

2.9303421710118944
2.9303421710118944


In [99]:
mean_squared_error(target,predicted,squared=False)
#len(target)

2.8226232082117217